## Binance top 200 coins by volume and momentum

In [1]:
coins = 150

## 1- Obtengo la data usando la API de Binance

In [4]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_top_volume_coins_list_api(count=200):
    # Obtener la lista de todos los símbolos de trading
    symbols_url = "https://api.binance.com/api/v3/exchangeInfo"
    symbols_response = requests.get(symbols_url)
    symbols_data = symbols_response.json()
    
    # Filtrar solo los pares con USDT
    usdt_symbols = [symbol['symbol'] for symbol in symbols_data['symbols'] if symbol['symbol'].endswith('USDT')]

    # Elimino las stablecoins
    stablecoins = ['USDCUSDT', 'FDUSDUSDT', '1000SATS', '1000SATSBTC', 'WBTC','WBTCUSDT', 'WETHUSDT']
    usdt_symbols = [symbol for symbol in usdt_symbols if symbol not in stablecoins]
    
    # Calcular la fecha de inicio (7 días atrás)
    start_time = int((datetime.now() - timedelta(days=7)).timestamp() * 1000)
    
    # Obtener el volumen para cada símbolo
    volumes = []
    for symbol in usdt_symbols:
        klines_url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d&startTime={start_time}"
        klines_response = requests.get(klines_url)
        klines_data = klines_response.json()
        
        # Calcular el volumen promedio
        total_volume = sum(float(kline[7]) for kline in klines_data) #7 es quote asset volume
        avg_volume = total_volume / len(klines_data) if klines_data else 0
        volumes.append((symbol[:-4], avg_volume))  # Removemos 'USDT' del símbolo
    
    # Ordenar por volumen descendente y tomar los primeros 'count'
    top_volumes = sorted(volumes, key=lambda x: x[1], reverse=True)[:count]
    
    return top_volumes  # Ahora retornamos la lista completa de tuplas (moneda, volumen)

def get_top_volume_coins_df_api(count=200):
    volumes = get_top_volume_coins_list_api(count)
    df = pd.DataFrame(volumes, columns=['coin', 'average volume last 7 days'])

    # Ordenar el DataFrame antes de formatear los números
    df = df.sort_values('average volume last 7 days', ascending=False)

   # Formatear los números después de ordenar
    df['average volume last 7 days'] = df['average volume last 7 days'].apply(lambda x: f'{x:.2f}')

    return df

In [8]:
# Esta lista de tuplas con los volumenes se guarda en la BD
api_top_volume_coins_list = get_top_volume_coins_list_api(coins)
api_top_volume_coins_list

[('BTC', 980586168.9311401),
 ('ETH', 438517208.77666986),
 ('SOL', 317744865.1110272),
 ('NEIRO', 270918179.7951902),
 ('PEPE', 266711009.88380122),
 ('SUI', 224806355.20482144),
 ('WIF', 174604521.83472428),
 ('XRP', 115173773.46234286),
 ('BNB', 92307544.78785715),
 ('OG', 90790938.9582),
 ('DOGE', 70783648.20954286),
 ('TAO', 69502875.15836285),
 ('EIGEN', 63160911.50568286),
 ('SHIB', 62320944.192315295),
 ('SEI', 62270233.35375857),
 ('APT', 62056439.82344286),
 ('FTT', 59780656.68391572),
 ('FET', 58857238.149285726),
 ('BONK', 54124645.09039395),
 ('WLD', 48910037.61408572),
 ('FTM', 47168548.07348572),
 ('1000SATS', 43009769.67909355),
 ('RUNE', 42883423.182314284),
 ('NEAR', 41223200.95631429),
 ('SAGA', 39887572.14538429),
 ('PEOPLE', 38270113.58847014),
 ('FLOKI', 37791528.60847347),
 ('AVAX', 35284594.09538571),
 ('DOGS', 33642498.31425321),
 ('ENA', 32052978.812898856),
 ('CATI', 32023535.137119997),
 ('TRX', 31262831.35430143),
 ('NOT', 30996200.399330284),
 ('HMSTR', 30

In [5]:
#api_top_coins_df = get_top_volume_coins_df_api(coins)
#api_top_coins_df

In [9]:
# Get the ticker list for coins. sin el volumen,
top_volume_coins = [item[0] for item in api_top_volume_coins_list]
#top_volume_coins

Uso la lista manual que estoy manejando en django

In [10]:
import os

# Obtener el directorio actual
path_actual = os.getcwd()

# Construir la ruta al archivo en la carpeta 'crypto_portfolio'
ruta_archivo = os.path.join(path_actual, 'crypto_portfolio', 'nombre_del_archivo.py')

print("Ruta del archivo:", ruta_archivo)

Ruta del archivo: /Users/egrados/Library/CloudStorage/OneDrive-Personal/Github/CS50-web-programming/5-capstone/crypto_portfolio/nombre_del_archivo.py


In [11]:
from crypto_portfolio.database import tickers_list


## 2- Obtener klines para cada criptomoeda

#### Nueva version que guarda los datos de dos semanas PASADA Y ACTUAL, tanto en CSV 

- Guarda los datos en la carpeta data_past2weeks/coinBTC.csv
- NO Guarda los datos en la base de datos, en la otra version lo hace.

In [14]:
# Obtiene klines con la lista top_coins, el par BTC y el par USDT de las ultimas 2 semanas

import pandas as pd
from datetime import datetime, timedelta, timezone
import os
import requests

not_available_btcpair = []

def get_klines_btcpair_past2weeks(crypto_list):
    today = datetime.now(timezone.utc)
    
    # Calcular el lunes de la semana actual
    monday_this_week = today - timedelta(days=today.weekday())
    
    # Calcular el lunes de hace dos semanas (inicio del período)
    start_monday = monday_this_week - timedelta(weeks=2)
    start_monday_at_midnight = start_monday.replace(hour=0, minute=0, second=0, microsecond=0)
    
    # Calcular el domingo de la semana pasada (fin del período)
    end_sunday = monday_this_week - timedelta(days=1)
    end_sunday_at_midnight = end_sunday.replace(hour=23, minute=59, second=59, microsecond=999999)

    # Convertir a timestamp en UTC
    start_timestamp = str(int(start_monday_at_midnight.timestamp() * 1000))
    end_timestamp = str(int(end_sunday_at_midnight.timestamp() * 1000))

    print(f"Periodo de obtención de datos: {start_monday_at_midnight} a {end_sunday_at_midnight}")

    dataframes = []

    # Crear la carpeta 'data' si no existe
    if not os.path.exists('data_past2weeks'):
        os.makedirs('data_past2weeks')

    for crypto in crypto_list:
        print(f"Obteniendo datos para {crypto}BTC...")
        symbol = f"{crypto}BTC"

        klines_url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d&startTime={start_timestamp}&endTime={end_timestamp}"
        klines_response = requests.get(klines_url)

        if klines_response.status_code == 200:
            klines_data = klines_response.json()
            if len(klines_data) == 0:
                print(f"No hay datos disponibles para {crypto}BTC")
                not_available_btcpair.append(crypto)
                continue

            # Crear el DataFrame
            df = pd.DataFrame(klines_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
            
            # Convertir el timestamp a datetime
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            
            # Establecer el timestamp como índice
            df.set_index('timestamp', inplace=True)
            
            # Convertir las columnas necesarias a float
            for col in ['open', 'high', 'low', 'close', 'volume']:
                df[col] = df[col].astype(float)
            
            # Eliminar las columnas que no necesitamos
            df = df.drop(['close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'], axis=1)

            # Guardar el DataFrame como CSV
            filename = f"data_past2weeks/{crypto}BTC.csv"
            df.to_csv(filename)
            
            dataframes.append(df)
            print(f"Datos de {crypto}BTC guardados en {filename}")
            print('-' * 50)
        
        elif klines_response.status_code in [404, 400]:
            print(f"Error: {crypto}BTC no está disponible en Binance.")
            not_available_btcpair.append(crypto)
            print('-' * 50)

    return dataframes

In [15]:
twoweeks_klines = get_klines_btcpair_past2weeks(top_volume_coins) #dos semanas atras, rebalnce anterior


Periodo de obtención de datos: 2024-09-23 00:00:00+00:00 a 2024-10-06 23:59:59.999999+00:00
Obteniendo datos para BTCBTC...
Error: BTCBTC no está disponible en Binance.
--------------------------------------------------
Obteniendo datos para ETHBTC...
Datos de ETHBTC guardados en data_past2weeks/ETHBTC.csv
--------------------------------------------------
Obteniendo datos para SOLBTC...
Datos de SOLBTC guardados en data_past2weeks/SOLBTC.csv
--------------------------------------------------
Obteniendo datos para NEIROBTC...
Error: NEIROBTC no está disponible en Binance.
--------------------------------------------------
Obteniendo datos para PEPEBTC...
Error: PEPEBTC no está disponible en Binance.
--------------------------------------------------
Obteniendo datos para SUIBTC...
Datos de SUIBTC guardados en data_past2weeks/SUIBTC.csv
--------------------------------------------------
Obteniendo datos para WIFBTC...
Datos de WIFBTC guardados en data_past2weeks/WIFBTC.csv
-------------

## 3. Data Analysis

Ahora tengo que modificar esta funcion para que :
- Solamente haga el analisis para la semana pasada y no la antepasada. 
- Anyada la columna normalized_close. 
- Haga el calculo del slope de normalized.
- Retorne el df ordenado de mayor a menor slope normalizado.



In [14]:
# OUTDATED VERSION PARA UNA SEMANA
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def analyze_crypto_data(data_folder='data_2'):
    results = []
    
    for filename in os.listdir(data_folder):
        if filename.endswith('.csv'):
            coin = filename.split('.')[0]
            filepath = os.path.join(data_folder, filename)
            
            # Leer el archivo CSV
            df = pd.read_csv(filepath)
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            # Calcular el retorno
            first_open = df.iloc[0]['open']
            last_close = df.iloc[-1]['close']
            returns = (last_close - first_open) / first_open * 100
            
            # Preparar datos para la regresión lineal
            X = np.arange(len(df)).reshape(-1, 1)
            y = df['close'].values
            
            # Realizar regresión lineal
            model = LinearRegression()
            model.fit(X, y)
            
            intercept = model.intercept_
            slope = model.coef_[0]
            
            results.append({
                'coin': coin,
                'return': returns,
                'intercept': intercept,
                'slope': slope
            })
            print(f"Análisis para {coin} completado.")
    
    # Crear DataFrame con los resultados
    result_df = pd.DataFrame(results)
    
    # Ordenar por slope de mayor a menor
    result_df = result_df.sort_values('slope', ascending=False)
    
    return result_df

# Ejemplo de uso
#df_maestro = analyze_crypto_data()
#df_maestro

#### Version normalizada y analiza seamana pasada y antepasada.

In [17]:
# Version que falla al encontrar datos faltantes, se corrige mas abajo.
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

def analyze_crypto_data_normalized(data_folder='data_past2weeks'):
    results_antepast_week = [] #results_week1
    results_past_week = [] #results_week2
    
    for filename in os.listdir(data_folder):
        if filename.endswith('.csv'):
            coin = filename.split('.')[0]
            print(f"Analizando datos de {coin}...")
            filepath = os.path.join(data_folder, filename)
            
            # Leer el archivo CSV
            df = pd.read_csv(filepath)
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            # Asegurarse de que el DataFrame esté ordenado por fecha
            df = df.sort_values('timestamp')
            
            # Dividir los datos en dos semanas
            past_week = df.iloc[-7:].copy()  # Semana Pasada - week2
            antepast_week = df.iloc[-14:-7].copy()  # Semana Antepasada - week1
            
            # Función para analizar una semana de datos
            def analyze_week(week_data):
                # Calcular el retorno
                first_open = week_data['open'].iloc[0]
                last_close = week_data['close'].iloc[-1]
                returns = (last_close - first_open) / first_open * 100
                
                # Normalizar los precios de cierre
                scaler = MinMaxScaler()
                week_data['normalized_close'] = scaler.fit_transform(week_data[['close']])
                
                # Preparar datos para la regresión lineal
                X = np.arange(len(week_data)).reshape(-1, 1)
                y = week_data['normalized_close'].values
                
                # Realizar regresión lineal
                model = LinearRegression()
                model.fit(X, y)
                
                intercept = model.intercept_
                slope = model.coef_[0]
                
                return {
                    'coin': coin,
                    'return': returns,
                    'intercept': intercept,
                    'slope': slope
                }
            
            # Analizar cada semana
            results_antepast_week.append(analyze_week(antepast_week))
            results_past_week.append(analyze_week(past_week))
            
            print(f"Análisis para {coin} completado.")
    
    # Crear DataFrames con los resultados
    result_df_antepast_week = pd.DataFrame(results_antepast_week)
    result_df_past_week = pd.DataFrame(results_past_week)
    
    # Ordenar por slope de mayor a menor
    result_df_week1 = result_df_week1.sort_values('slope', ascending=False)
    result_df_week2 = result_df_week2.sort_values('slope', ascending=False)
    
    return result_df_week1, result_df_week2

In [16]:
# Ejemplo de uso
#df_maestro2 = analyze_crypto_data_normalized()
#df_maestro2[1]

##### TROUBLEESHOOTING para cuando no tienes los datos completos de la semana anterior

In [17]:
# version que anyade 0s a la data faltante
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

def analyze_crypto_data_normalized(data_folder='data_2_bak'):
    results_week1 = []
    results_week2 = []
    
    for filename in os.listdir(data_folder):
        if filename.endswith('.csv'):
            coin = filename.split('.')[0]
            filepath = os.path.join(data_folder, filename)
            
            # Leer el archivo CSV
            df = pd.read_csv(filepath)
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            # Asegurarse de que el DataFrame esté ordenado por fecha
            df = df.sort_values('timestamp')
            
            # Verificar si hay 14 valores y completar si es necesario
            if len(df) < 14:
                last_date = df['timestamp'].max()
                missing_dates = pd.date_range(end=last_date, periods=14) - pd.Timedelta(days=13)
                missing_df = pd.DataFrame({
                    'timestamp': missing_dates,
                    'open': 0,
                    'high': 0,
                    'low': 0,
                    'close': 0,
                    'volume': 0
                })
                df = pd.concat([missing_df, df]).drop_duplicates(subset=['timestamp']).sort_values('timestamp')
                df = df.tail(14)
                print(f"Datos incompletos para {coin}. Se han rellenado los valores faltantes.")
            elif len(df) > 14:
                df = df.tail(14)
            
            # Dividir los datos en dos semanas
            week2 = df.iloc[-7:].copy()  # Última semana
            week1 = df.iloc[-14:-7].copy()  # Penúltima semana
            
            # Función para analizar una semana de datos
            def analyze_week(week_data):
                # Calcular el retorno
                first_open = week_data['open'].iloc[0]
                last_close = week_data['close'].iloc[-1]
                returns = (last_close - first_open) / first_open * 100 if first_open != 0 else 0
                
                # Normalizar los precios de cierre
                scaler = MinMaxScaler()
                week_data['normalized_close'] = scaler.fit_transform(week_data[['close']])
                
                # Preparar datos para la regresión lineal
                X = np.arange(len(week_data)).reshape(-1, 1)
                y = week_data['normalized_close'].values
                
                # Realizar regresión lineal
                model = LinearRegression()
                model.fit(X, y)
                
                intercept = model.intercept_
                slope = model.coef_[0]
                
                return {
                    'coin': coin,
                    'return': returns,
                    'intercept': intercept,
                    'slope': slope
                }
            
            # Analizar cada semana
            results_week1.append(analyze_week(week1))
            results_week2.append(analyze_week(week2))
            
            print(f"Análisis para {coin} completado.")
    
    # Crear DataFrames con los resultados
    result_df_week1 = pd.DataFrame(results_week1)
    result_df_week2 = pd.DataFrame(results_week2)
    
    # Ordenar por slope de mayor a menor
    result_df_week1 = result_df_week1.sort_values('slope', ascending=False)
    result_df_week2 = result_df_week2.sort_values('slope', ascending=False)
    
    return result_df_week1, result_df_week2

In [18]:
# Ejemplo de uso
#week 2 es ultima semana y week 1 es antepenultima semana
df_maestro2_debug = analyze_crypto_data_normalized()


Análisis para MINABTC completado.
Análisis para HIGHBTC completado.
Análisis para AGLDBTC completado.
Análisis para LSKBTC completado.
Análisis para WLDBTC completado.
Análisis para ANKRBTC completado.
Análisis para VIDTBTC completado.
Análisis para VITEBTC completado.
Análisis para CELRBTC completado.
Análisis para RADBTC completado.
Análisis para GLMBTC completado.
Análisis para CFXBTC completado.
Análisis para FLMBTC completado.
Análisis para AXSBTC completado.
Análisis para BLURBTC completado.
Análisis para OXTBTC completado.
Análisis para CRVBTC completado.
Análisis para LDOBTC completado.
Análisis para APEBTC completado.
Análisis para BNBBTC completado.
Análisis para COMPBTC completado.
Análisis para NFPBTC completado.
Análisis para RLCBTC completado.
Análisis para LINKBTC completado.
Análisis para HFTBTC completado.
Análisis para RIFBTC completado.
Análisis para YGGBTC completado.
Análisis para CTSIBTC completado.
Análisis para ENJBTC completado.
Análisis para YFIBTC completado.

In [19]:
# semana pasada
df_maestro2_debug_pastweek = df_maestro2_debug[1]
vvv = df_maestro2_debug_pastweek['coin']

In [20]:
print(vvv)

166      DODOBTC
119       BLZBTC
170       FISBTC
60        DIABTC
269      CELOBTC
         ...    
204       TONBTC
52        TRUBTC
6        VIDTBTC
49     BANANABTC
36        BNXBTC
Name: coin, Length: 272, dtype: object


In [21]:
# semana antepasada
df_maestro2_debug_antepastweek = df_maestro2_debug[0]
df_maestro2_debug_antepastweek

,coin,return,intercept,slope
80,POLBTC,0.000000,-0.208710,0.210814
35,BNTBTC,0.000000,0.179789,0.176831
105,METISBTC,0.000000,0.168274,0.174531
101,COTIBTC,28.859060,0.095779,0.167208
79,ROSEBTC,8.602151,0.022321,0.165179
...,...,...,...,...
48,QKCBTC,-12.500000,1.250000,-0.178571
6,VIDTBTC,-22.826087,0.957792,-0.178571
121,ELFBTC,-6.373293,1.185961,-0.183498
59,TLMBTC,-10.000000,1.071429,-0.214286


##### Version que añade R2 como parametro y esta normalizado y añade 0s y analiza las dos semanas

In [39]:
# VERSION QUE USA R2
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

def analyze_crypto_data_r2(data_folder='data_past2weeks'):
    results_week1 = []
    results_week2 = []
    
    for filename in os.listdir(data_folder):
        if filename.endswith('.csv'):
            coin = filename.split('.')[0]
            filepath = os.path.join(data_folder, filename)
            
            # Leer el archivo CSV
            df = pd.read_csv(filepath)
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            # Asegurarse de que el DataFrame esté ordenado por fecha
            df = df.sort_values('timestamp')
            
            # Verificar si hay 14 valores y completar si es necesario
            if len(df) < 14:
                last_date = df['timestamp'].max()
                missing_dates = pd.date_range(end=last_date, periods=14) - pd.Timedelta(days=13)
                missing_df = pd.DataFrame({
                    'timestamp': missing_dates,
                    'open': 0,
                    'high': 0,
                    'low': 0,
                    'close': 0,
                    'volume': 0
                })
                df = pd.concat([missing_df, df]).drop_duplicates(subset=['timestamp']).sort_values('timestamp')
                df = df.tail(14)
                print(f"Datos incompletos para {coin}. Se han rellenado los valores faltantes.")
            elif len(df) > 14:
                df = df.tail(14)
            
            # Dividir los datos en dos semanas
            week2 = df.iloc[-7:].copy()  # Última semana
            week1 = df.iloc[-14:-7].copy()  # Penúltima semana
            
            # Función para analizar una semana de datos
            def analyze_week(week_data):
                # Calcular el retorno
                first_open = week_data['open'].iloc[0]
                last_close = week_data['close'].iloc[-1]
                returns = (last_close - first_open) / first_open * 100 if first_open != 0 else 0
                
                # Normalizar los precios de cierre
                scaler = MinMaxScaler()
                week_data['normalized_close'] = scaler.fit_transform(week_data[['close']])
                
                # Preparar datos para la regresión lineal
                X = np.arange(len(week_data)).reshape(-1, 1)
                y = week_data['normalized_close'].values
                
                # Realizar regresión lineal
                model = LinearRegression()
                model.fit(X, y)
                
                intercept = model.intercept_
                slope = model.coef_[0]
                
                # Calcular R2
                r2 = r2_score(y, model.predict(X))
                
                return {
                    'coin': coin,
                    'return': returns,
                    'intercept': intercept,
                    'slope': slope,
                    'r2': r2
                }
            
            # Analizar cada semana
            results_week1.append(analyze_week(week1))
            results_week2.append(analyze_week(week2))
            
            print(f"Análisis para {coin} completado.")
    
    # Crear DataFrames con los resultados
    result_df_week1 = pd.DataFrame(results_week1)
    result_df_week2 = pd.DataFrame(results_week2)
    
    # Filtrar y ordenar los resultados
    def filter_and_sort(df):
        df_filtered = df[df['r2'] >= 0.9]
        return df_filtered.sort_values('slope', ascending=False)
    
    result_df_week1 = filter_and_sort(result_df_week1)
    result_df_week2 = filter_and_sort(result_df_week2)
    
    # Asegurarse de que R2 esté incluido en los resultados finales
    columns_order = ['coin', 'return', 'slope', 'intercept', 'r2']
    result_df_week1 = result_df_week1[columns_order]
    result_df_week2 = result_df_week2[columns_order]
    
    return result_df_week1, result_df_week2

In [40]:
# Ejemplo de uso
#week 2 es ultima semana y week 1 es antepenultima semana
df_maestro2_debug_r2 = analyze_crypto_data_r2()

Análisis para MINABTC completado.
Análisis para HIGHBTC completado.
Análisis para AGLDBTC completado.
Análisis para LSKBTC completado.
Análisis para WLDBTC completado.
Análisis para ANKRBTC completado.
Análisis para VIDTBTC completado.
Análisis para VITEBTC completado.
Análisis para CELRBTC completado.
Análisis para RADBTC completado.
Análisis para GLMBTC completado.
Análisis para CFXBTC completado.
Análisis para FLMBTC completado.
Análisis para AXSBTC completado.
Análisis para BLURBTC completado.
Análisis para OXTBTC completado.
Análisis para CRVBTC completado.
Análisis para LDOBTC completado.
Análisis para APEBTC completado.
Análisis para BNBBTC completado.
Análisis para COMPBTC completado.
Análisis para NFPBTC completado.
Análisis para RLCBTC completado.
Análisis para LINKBTC completado.
Análisis para HFTBTC completado.
Análisis para RIFBTC completado.
Análisis para YGGBTC completado.
Análisis para CTSIBTC completado.
Análisis para ENJBTC completado.
Análisis para YFIBTC completado.

In [42]:
# semana pasada
df_maestro2_debug_pastweek = df_maestro2_debug_r2[1]
vvv = df_maestro2_debug_pastweek
vvv

,coin,return,slope,intercept,r2
170,FISBTC,0.559701,0.190154,-0.087838,0.913755
60,DIABTC,3.315881,0.190027,-0.068733,0.938669
182,TAOBTC,44.722929,0.179777,-0.055278,0.948239
106,IMXBTC,22.643253,0.161007,0.060044,0.944454
165,SUIBTC,36.677632,0.157692,0.070311,0.964213
98,INJBTC,8.209192,0.144201,0.105240,0.915399
93,DGBBTC,-9.090909,0.000000,0.000000,1.000000
184,RAREBTC,-13.502110,-0.148352,0.884615,0.902228
36,BNXBTC,-22.489796,-0.176647,1.145291,0.901848


In [45]:
# semana antepasada
df_maestro2_debug_antepastweek = df_maestro2_debug_r2[0]
yyy = df_maestro2_debug_antepastweek
yyy


,coin,return,slope,intercept,r2
101,COTIBTC,28.859060,0.167208,0.095779,0.945207
271,NTRNBTC,0.000000,0.000000,0.000000,1.000000
173,MATICBTC,0.000000,0.000000,0.000000,1.000000
31,PROMBTC,0.000000,0.000000,0.000000,1.000000
69,KLAYBTC,-6.779661,-0.149160,0.892857,0.910585
139,RDNTBTC,-13.286713,-0.151786,1.019643,0.964877
237,OAXBTC,-9.677419,-0.152857,0.944286,0.953448
187,AUDIOBTC,-7.929515,-0.154018,0.953125,0.909977
64,XTZBTC,-5.683837,-0.157834,1.047235,0.913333
190,MLNBTC,-10.119695,-0.160377,0.958221,0.902102


In [46]:
# ULTIMA VERSION QUE ELIMINA LOS QUE TIENEN 0 COMO VALORES, no funciona.
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

def analyze_crypto_data_r2_no0(data_folder='data_2'):
    results_week1 = []
    results_week2 = []
    
    for filename in os.listdir(data_folder):
        if filename.endswith('.csv'):
            coin = filename.split('.')[0]
            filepath = os.path.join(data_folder, filename)
            
            # Leer el archivo CSV
            df = pd.read_csv(filepath)
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            # Asegurarse de que el DataFrame esté ordenado por fecha
            df = df.sort_values('timestamp')
            
            # Verificar si hay 14 valores y completar si es necesario
            if len(df) < 14:
                last_date = df['timestamp'].max()
                missing_dates = pd.date_range(end=last_date, periods=14) - pd.Timedelta(days=13)
                missing_df = pd.DataFrame({
                    'timestamp': missing_dates,
                    'open': 0,
                    'high': 0,
                    'low': 0,
                    'close': 0,
                    'volume': 0
                })
                df = pd.concat([missing_df, df]).drop_duplicates(subset=['timestamp']).sort_values('timestamp')
                df = df.tail(14)
                print(f"Datos incompletos para {coin}. Se han rellenado los valores faltantes.")
            elif len(df) > 14:
                df = df.tail(14)
            
            # Dividir los datos en dos semanas
            week2 = df.iloc[-7:].copy()  # Última semana
            week1 = df.iloc[-14:-7].copy()  # Penúltima semana
            
            # Función para analizar una semana de datos
            def analyze_week(week_data):
                # Verificar si hay datos rellenados (ceros) en esta semana
                if (week_data[['open', 'high', 'low', 'close', 'volume']] == 0).any().any():
                    return None  # No analizar esta semana si hay datos rellenados
                
                # Calcular el retorno
                first_open = week_data['open'].iloc[0]
                last_close = week_data['close'].iloc[-1]
                returns = (last_close - first_open) / first_open * 100
                
                # Normalizar los precios de cierre
                scaler = MinMaxScaler()
                week_data['normalized_close'] = scaler.fit_transform(week_data[['close']])
                
                # Preparar datos para la regresión lineal
                X = np.arange(len(week_data)).reshape(-1, 1)
                y = week_data['normalized_close'].values
                
                # Realizar regresión lineal
                model = LinearRegression()
                model.fit(X, y)
                
                intercept = model.intercept_
                slope = model.coef_[0]
                
                # Calcular R2
                r2 = r2_score(y, model.predict(X))
                
                return {
                    'coin': coin,
                    'return': returns,
                    'intercept': intercept,
                    'slope': slope,
                    'r2': r2
                }
            
            # Analizar cada semana
            week1_result = analyze_week(week1)
            week2_result = analyze_week(week2)
            
            if week1_result:
                results_week1.append(week1_result)
            if week2_result:
                results_week2.append(week2_result)
            
            print(f"Análisis para {coin} completado.")
    
    # Crear DataFrames con los resultados
    result_df_week1 = pd.DataFrame(results_week1)
    result_df_week2 = pd.DataFrame(results_week2)
    
    # Filtrar y ordenar los resultados
    def filter_and_sort(df):
        if not df.empty:
            df_filtered = df[df['r2'] >= 0.9]
            return df_filtered.sort_values('slope', ascending=False)
        return df
    
    result_df_week1 = filter_and_sort(result_df_week1)
    result_df_week2 = filter_and_sort(result_df_week2)
    
    # Asegurarse de que R2 esté incluido en los resultados finales
    columns_order = ['coin', 'return', 'slope', 'intercept', 'r2']
    if not result_df_week1.empty:
        result_df_week1 = result_df_week1[columns_order]
    if not result_df_week2.empty:
        result_df_week2 = result_df_week2[columns_order]
    
    return result_df_week1, result_df_week2

In [49]:
df_maestro2_debug_r2 = analyze_crypto_data_r2_no0()

Análisis para ETHBTC completado.
Análisis para BCHBTC completado.
Análisis para SUIBTC completado.
Análisis para TAOBTC completado.
Análisis para ETHFIBTC completado.


In [50]:
# semana pasada
df_maestro2_debug_pastweek = df_maestro2_debug_r2[1]
vvv = df_maestro2_debug_pastweek
vvv

,coin,return,slope,intercept,r2
3,TAOBTC,44.722929,0.179777,-0.055278,0.948239
2,SUIBTC,36.677632,0.157692,0.070311,0.964213


In [51]:
# semana antepasada
df_maestro2_debug_antepastweek = df_maestro2_debug_r2[0]
yyy = df_maestro2_debug_antepastweek
yyy

,coin,return,intercept,slope,r2


## 4. Ploteo de grafica

In [22]:
# Ploteo 1, plotea bien el linechart pero no el candlestick

import os
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf

def plot_crypto_data(df_maestro, data_folder='data', plot_folder='plots'):
    # Crear la carpeta 'plots' si no existe
    if not os.path.exists(plot_folder):
        os.makedirs(plot_folder)

    #for _, row in df_maestro.iterrows():
    for _, row in df_maestro.head(6).iterrows():
        coin = row['coin']
        slope = row['slope']
        intercept = row['intercept']

        # Leer el archivo CSV correspondiente
        df = pd.read_csv(os.path.join(data_folder, f"{coin}.csv"))
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)

        # Calcular la línea de regresión
        x = range(len(df))
        regression_line = [slope * xi + intercept for xi in x]

        # Gráfico de línea
        plt.figure(figsize=(12, 6))
        plt.plot(df.index, df['close'], label='Precio de cierre')
        plt.plot(df.index, regression_line, label='Regresión lineal', color='red')
        plt.title(f"Análisis de {coin} - Gráfico de línea")
        plt.xlabel("Fecha")
        plt.ylabel("Precio")
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(plot_folder, f"{coin}_line.png"))
        plt.show()

        # Gráfico de velas (candlestick)
        fig, ax = plt.subplots(figsize=(12, 6))
        mpf.plot(df, type='candle', style='yahoo', ax=ax)
        ax.plot(df.index, regression_line, label='Regresión lineal', color='red')
        ax.set_title(f"Análisis de {coin} - Gráfico de velas")
        ax.set_xlabel("Fecha")
        ax.set_ylabel("Precio")
        ax.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(plot_folder, f"{coin}_candlestick.png"))
        plt.show()

# Ejemplo de uso:
#df_maestro = analyze_crypto_data()  # Asumiendo que esta función ya fue definida y ejecutada
plot_crypto_data(df_maestro)

NameError: name 'df_maestro' is not defined

In [47]:
asd = df_maestro.iterrows()
for _, row in df_maestro.head(10).iterrows():
    print (row['coin'])

TAOBTC
ETHBTC
AAVEBTC
BNBBTC
BCHBTC
SOLBTC
TRBBTC
METISBTC
AVAXBTC
ORDIBTC
